# Importación de librerias y de datos

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [17]:
pokemons = pd.read_csv("Pokemon.csv")

Se puede ver que la única columna con valores NaN es **Type 2**

In [18]:
pokemons.info()
pokemons

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


# Preprocesamiento de datos

- Se usan solo los pokemons de la primera generación.
- Se extraen los pokemons normales (base viene con estadisticas de versiones Mega).

In [19]:
primera_gen = pokemons[pokemons["Generation"]==1]
primera_gen = primera_gen[~primera_gen["Name"].str.contains("Mega")]
primera_gen_inicial = primera_gen.copy()
primera_gen = primera_gen.drop(["Generation"], axis=1)

- Se aplica normalización Min-Max sobre los datos númericos.

In [20]:
from sklearn.preprocessing import MinMaxScaler

numerical_cols = ["Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]

scaler = MinMaxScaler()
scaled = scaler.fit_transform(primera_gen[numerical_cols])
primera_gen[numerical_cols] = scaled


- Se transforman los valores booleanos a 0 y 1.
- Se rellena los valores NaN en la columna de **Type 2** con el string "None".

In [21]:
primera_gen["Legendary"] = primera_gen["Legendary"].replace({True: 1, False: 0})
primera_gen["Type 2"] = primera_gen["Type 2"].fillna("None")

### Preprocesamiento de categorias (Types)
- Por cada tipo distinto se asocia un número.
- Se codifica usando One-Hot.
- Se genera así una nueva columna ie. *feature* por cada tipo distinto que toma valores binarios
- Se hace tanto para **Type 1** como para **Type 2**.

In [22]:
from sklearn.preprocessing import LabelBinarizer

encoder_1 = LabelBinarizer()
type_1_encoded = encoder_1.fit_transform(primera_gen["Type 1"])
encoder_2 = LabelBinarizer()
type_2_encoded = encoder_2.fit_transform(primera_gen["Type 2"])

# Se agrega " 2" a los types 2 para distinguirlos de los type 1
cols_names_type_2 = np.char.add(encoder_2.classes_, " 2")

#Se agregan las codificaciones One-Hot al DataFrame
primera_gen[encoder_1.classes_] = type_1_encoded
primera_gen[cols_names_type_2] = type_2_encoded

- Se eliminan las columnas que no se van a usar

In [23]:
primera_gen.reset_index(drop=True, inplace=True)
primera_gen_init = primera_gen.copy()

primera_gen.drop(["Type 1"], axis = 1, inplace=True)
primera_gen.drop(["Type 2"], axis = 1, inplace=True)
primera_gen.drop(["Name"], axis = 1, inplace=True)
primera_gen.drop(["#"], axis = 1, inplace=True)

primera_gen

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Bug,Dragon,...,Flying 2,Grass 2,Ground 2,Ice 2,None 2,Poison 2,Psychic 2,Rock 2,Steel 2,Water 2
0,0.253608,0.145833,0.341085,0.251429,0.359712,0.428571,0.24,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.432990,0.208333,0.441860,0.331429,0.467626,0.571429,0.36,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0.680412,0.291667,0.596899,0.445714,0.611511,0.761905,0.52,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0.235052,0.120833,0.364341,0.217143,0.323741,0.285714,0.40,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.432990,0.200000,0.457364,0.302857,0.467626,0.428571,0.52,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.216495,0.129167,0.457364,0.228571,0.251799,0.285714,0.28,0,0,1,...,0,0,0,0,1,0,0,0,0,0
147,0.463918,0.212500,0.612403,0.342857,0.395683,0.476190,0.44,0,0,1,...,0,0,0,0,1,0,0,0,0,0
148,0.835052,0.337500,1.000000,0.514286,0.611511,0.761905,0.52,0,0,1,...,1,0,0,0,0,0,0,0,0,0
149,1.000000,0.400000,0.813953,0.485714,1.000000,0.666667,0.92,1,0,0,...,0,0,0,0,1,0,0,0,0,0


# T-SNE de Scikit-Learn

- Se importa y declara t-SNE con dos componentes para graficar.
- Se entrena el modelo con los datos preprocesados.

In [30]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, init="pca", learning_rate="auto", random_state = 42)
primera_gen_2d = tsne.fit_transform(primera_gen.to_numpy())

# Visualización de resultados

- Se usa la librería Plotly para generar gráficos interactivos.
- Se crea DataFrame para visualización, uniendo *primera_gen_inicial* con los componentes extraidos usando t-SNE
- Se usa *primera_gen_inicial* porque tiene los valores originales de la primera generación, antes del preprocesamiento.
- Se colorea según **tipo primario** del Pokemon.

In [67]:
df_tsne = pd.DataFrame(primera_gen_2d, columns = ["t-SNE dimension 1", "t-SNE dimension 2"])

primera_gen_inicial.reset_index(drop=True, inplace=True)# Para que calcen las filas de t-SNE con las de primera_gen_inicial
primera_gen_inicial["Type 2"] = primera_gen_inicial["Type 2"].fillna("None")
df = pd.concat([primera_gen_inicial, df_tsne], axis=1)# Concatenar columna-wise

# hover_data: Atributos que se muestran en lista pop up al colocar cursor sobre datapoint
# hover_name: Para el titulo del pop up
# color: Columna segun la que se colorea
fig = px.scatter(df, "t-SNE dimension 1", "t-SNE dimension 2", hover_data = primera_gen_inicial.columns, hover_name = "Name", color = "Type 1")

# Para graficar con nombres descomentar linea siguiente
#fig = px.scatter(df, "t-SNE dimension 1", "t-SNE dimension 2", hover_data = primera_gen_inicial.columns, hover_name = "Name", text = "Name", color = "Type 1")

# Para colocar titulo centrado
fig.update_layout(
    title=dict(
        text="Visualización de Pokemon usando t-SNE <br> Coloreados por tipo principal",
        font=dict(size=22),
        x=0.5,
        xref="paper"
    )
)

fig.write_image("images/by_type.svg") # Guarda imagen como .svg
fig.write_html("html/by_type.html") # Guarda grafico interactivo como .html
fig.show()


# Usando Tipo Secundario para colorear

- Idem a gráfico anterior pero coloreando esta vez según **tipo secundario**.

In [70]:
fig = px.scatter(df, "t-SNE dimension 1", "t-SNE dimension 2", hover_data = primera_gen_inicial.columns, hover_name = "Name", color = "Type 2")
# Para graficar con nombres descomentar linea siguiente
#fig = px.scatter(df, "t-SNE dimension 1", "t-SNE dimension 2", hover_data = primera_gen_inicial.columns, hover_name = "Name", text = "Name", color = "Type 2")

fig.update_layout(
    title=dict(
        text="Visualización de Pokemon usando t-SNE <br> Coloreados por tipo secundario",
        font=dict(size=22),
        x=0.5,
        xref="paper"
    )
)
fig.write_image("images/by_type_2.svg")
fig.write_html("html/by_type_2.html")
fig.show()

# Gráfico 3D

- Plotly permite graficar en 3D facilmente.
- Se entrena el algoritmo t-SNE con 3 componentes.
- Se visualizan resultados, coloreando por **tipo principal** y con tamaño de esfera relativa al **poder total**.
- **Poder total** hace referencia a la suma de todas las estadisticas (ataque, defensa, etc).


In [71]:
tsne_3 = TSNE(n_components=3, init="pca", learning_rate="auto", random_state = 42)
primera_gen_3d = tsne_3.fit_transform(primera_gen.to_numpy())
df_tsne_3 = pd.DataFrame(primera_gen_3d, columns = ["t-SNE dimension 1", "t-SNE dimension 2", "t-SNE dimension 3"])

df_3d = pd.concat([primera_gen_inicial, df_tsne_3], axis=1)
fig = px.scatter_3d(df_3d, "t-SNE dimension 1", "t-SNE dimension 2", "t-SNE dimension 3", hover_data = primera_gen_inicial.columns, hover_name = "Name", text = "Name", color = "Type 1", size = "Total")


fig.update_layout(
    title=dict(
        text="Visualización 3D de Pokemon usando t-SNE <br> Coloreados por tipo principal y tamaño por poder total",
        font=dict(size=22),
        x=0.5,
        xref="paper"
    )
)

fig.write_html("html/pokemon_3d.html")